Downsampling examples

In [2]:
import numpy as np
import SimpleITK as sitk

from skimage.transform import rescale, resize, downscale_local_mean

In [8]:
# Resample with simple itk
# see https://gist.github.com/mrajchl/ccbd5ed12eb68e0c1afc5da116af614a#file-resample_itk_image-py-L6
def resample_img_itk(itk_image, out_spacing=[2.0, 2.0, 2.0], nearest=False):
    
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if nearest:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

# use downscale_local_mean from scikit-image
def resample_arr_skimg(np_array, in_spacing=[1.0,1.0,1.0], factors=[4,4,4]):
    out_spacing = np.array(in_spacing) * np.array(factors)
    image_downscaled = downscale_local_mean(np_array, tuple(factors), cval=0.0)
    itk_image = sitk.GetImageFromArray(image_downscaled)
    itk_image.SetOrigin((0, 0, 0))
    itk_image.SetSpacing(out_spacing)
    return itk_image

In [4]:
# Downsample jrc2018
jrc18_path='sampleImages/JRC2018_FEMALE_38um_iso_16bit.nrrd'
jrc18 = sitk.ReadImage(jrc18_path)
jrc18.GetSpacing()

# this spacing is equivalent to 4x downsampling
jrc18_down = resample_img_itk( jrc18, out_spacing=[1.52,1.52,1.52])
writer = sitk.ImageFileWriter()
writer.SetFileName('sampleImages/jrc18_down.nrrd')
writer.Execute(jrc18_down)

In [9]:
# Downsample jrc2018 using local mean
factors = [4,4,4]
in_spacing = jrc18.GetSpacing()
ski_down = resample_arr_skimg( sitk.GetArrayFromImage( jrc18 ).astype(np.float32), in_spacing=in_spacing, factors=factors)

writer = sitk.ImageFileWriter()
writer.SetFileName('sampleImages/jrc18_down_avg.nrrd')
writer.Execute(ski_down)

Now we have these three images in the `sampleImages` folder. Open them together in your favorite viewer (I'll use BigDataViewer)

* `JRC2018_FEMALE_38um_iso_16bit.nrrd`
* `jrc18_down.nrrd`
* `jrc18_down_avg.nrrd`

What do you notice?

In [6]:
# Downsample jrc2010
jrc10_path='sampleImages/JFRCtemplate2010.nrrd'
jrc10 = sitk.ReadImage(jrc10_path)

(0.622088, 0.622088, 0.622088)

In [7]:
# this spacing is equivalent to 4x downsampling
jrc10_down = resample_img_itk( jrc10, out_spacing=[2.488352,2.488352,2.488352])
writer = sitk.ImageFileWriter()
writer.SetFileName('jrc10_down.nrrd')
writer.Execute(jrc10_down)